# Example of an one token economy

This notebook shows a very simple example of how to setup an one token economy within TokenLab. That's the most basic type of token economy. The assumptions are the following:

1. There is only one token. We are modelling this using the TokenEconomy_Basic class
2. 

In [1]:
import numpy as np
import scipy
import sys
sys.path.insert(0,'..')

from simulationcomponents import *
from simulationcomponents.usergrowthclasses import *
from simulationcomponents.transactionclasses import *
from simulationcomponents.tokeneconomyclasses import *
from simulationcomponents.transactionclasses import *
from simulationcomponents.agentpoolclasses import *
from simulationcomponents.pricingclasses import *
from utils.helpers import *
from matplotlib import pyplot as plt

# Globals

In [2]:
ITERATIONS=72
HOLDING_TIME=0.1
SUPPLY=0.17*10**9

## Luxury digital art fractional marketplace

In [3]:
#Primary
usm_fiat=UserGrowth_Spaced(10,54,ITERATIONS,log_saturated_space)
tsm_fiat=TransactionManagement_Stochastic(activity_probs=np.linspace(0.25,0.1,ITERATIONS),
                                            value_dist_parameters={'loc':10000,'scale':10000/7},
                                           transactions_dist_parameters={'mu':1})

ap_fiat=AgentPool_IndependentUserGrowth(users_controller=usm_fiat,transactions_controller=tsm_fiat,currency='$')

In [4]:
#Secondary
#make the sequence of parameters
loc_param=generate_distribution_param_from_sequence('loc',np.linspace,500,750,ITERATIONS)
scale_param=generate_distribution_param_from_sequence('scale',np.linspace,500/7,750/7,ITERATIONS)
value_dist_params=merge_param_dists(loc_param,scale_param)


usm_token=UserGrowth_Spaced(10000,327800 ,ITERATIONS,log_saturated_space)
tsm_token=TransactionManagement_Stochastic(activity_probs=np.linspace(0.25,0.1,ITERATIONS),
                                            value_dist_parameters=value_dist_params,
                                           transactions_dist_parameters={'mu':26/12})

ap_token=AgentPool_IndependentUserGrowth(users_controller=usm_token,transactions_controller=tsm_token,currency='tokenA')

# Token economy

In [5]:
te=TokenEconomy_Basic(holding_time=HOLDING_TIME,supply=SUPPLY,token='tokenA')
te.add_agent_pools([ap_fiat,ap_token])

True

In [6]:
meta=TokenMetaSimulator(te)
meta.execute(iterations=ITERATIONS,repetitions=50)
reps=meta.get_data()

100%|███████████████████████████████████████████| 50/50 [00:01<00:00, 49.38it/s]


In [7]:
reps

,transactions_fiat,supply,holding_time,tokenA,tokenA_price,repetition_run,iteration_time
0,240622.059384,170000000.0,0.1,0.000000e+00,None,0,0
1,8464.474065,170000000.0,0.1,5.977608e+07,None,0,1
2,442.759401,170000000.0,0.1,8.892354e+07,None,0,2
3,61088.536256,170000000.0,0.1,2.957135e+07,None,0,3
4,52382.262860,170000000.0,0.1,5.900050e+07,None,0,4
...,...,...,...,...,...,...,...
67,1355.125672,170000000.0,0.1,1.310920e+01,None,49,67
68,60885.942985,170000000.0,0.1,1.856124e+07,None,49,68
69,4069.569922,170000000.0,0.1,5.520691e+07,None,49,69
70,86.824936,170000000.0,0.1,3.626978e+07,None,49,70


In [8]:
plot,data=meta.get_timeseries('price')
plot

KeyError: 'Column not found: price'

# Stochastic holding time

In [ ]:
te=TokenEconomy_Basic(holding_time=HoldingTime_Stochastic(),supply=SUPPLY,token='tokenA')
te.add_agent_pools([ap_luxury,ap_luxury_sec,ap_fine])
meta=TokenMetaSimulator(te)
meta.execute(iterations=ITERATIONS,repetitions=50)
reps=meta.get_data()
plot,data=meta.get_timeseries('price')
plot

# Price plus noise

In [ ]:
te=TokenEconomy_Basic(holding_time=HoldingTime_Stochastic(),supply=SUPPLY,price_function=PriceFunction_EOE_Noise,token='tokenA')
te.add_agent_pools([ap_luxury,ap_luxury_sec,ap_fine])
meta=TokenMetaSimulator(te)
meta.execute(iterations=ITERATIONS,repetitions=50)
reps=meta.get_data()
plot,data=meta.get_timeseries('price')
plot

# Price plus noise plus adaptive variance

In [ ]:
te=TokenEconomy_Basic(holding_time=HoldingTime_Stochastic(),supply=SUPPLY,
                      price_function=PriceFunction_EOE_AdaptiveNoise,token='tokenA')
te.add_agent_pools([ap_luxury,ap_luxury_sec,ap_fine])
meta=TokenMetaSimulator(te)
meta.execute(iterations=ITERATIONS,repetitions=50)
reps=meta.get_data()
plot,data=meta.get_timeseries('price')
plot